 # An Example of using `minimod`
 For this notebook, we will first create the data needs to be used for the program. Then we will instantiate the class and make a report of the results using the `report` method

In [1]:
import sys
import pandas as pd

import minimod as mm



Using Python-MIP package version 1.6.8


 ## Data
 This is how the data was processed, assuming we use the Katie_VA_Benefits_and_Costs_1_8_2019.xlsx file.
 ```python
 excel_file = "/home/lordflaron/Documents/GAMS-Python/Cameroon VA/GAMS_Working/GAMS_R Project/Katie_VA_Benefits_and_Costs_1_8_2019.xlsx"
 df_benefit = (pd.read_excel(excel_file,
                            sheet_name = 'Benefits',
                            header = 2)
               .set_index(['intervention', 'space'])
               .stack()
               .to_frame()
               .reset_index()
               .rename({'level_2' : 'time',
                        0 : 'benefit'}, axis=1)
               .set_index(['intervention', 'space', 'time'])
               )
 df_cost = (pd.read_excel(excel_file,
                            sheet_name = 'Costs',
                            header = 2)
               .set_index(['intervention', 'space'])
               .stack()
               .to_frame()
               .reset_index()
               .rename({'level_2' : 'time',
                        0 : 'costs'}, axis=1)
               .set_index(['intervention', 'space', 'time'])
               )
 ```

 Then we merge the cost and benefit data together.
 ```python
 df = (df_benefit
       .merge(df_cost, left_index=True, right_index=True)
 )
```
 Then we save the data. The finished data can be found in the `/examples/data` folder.
 `df.to_csv("./data/example1.csv")`

In [3]:

df = pd.read_csv("data/example1.csv")

 ## Cost Minimization
 Now we instantiate the model, and then run `fit` and get the report.

In [3]:
c = mm.CostSolver(data = df)


In [4]:

opt = c.fit()


              Loading MIP Model with:
              Solver = CBC
              Method = MIN
              
[Note]: Optimal Solution Found


In [5]:

c.report()





              Optimized Scenario with:
              
              Method: MIN
              Discount Factor on Costs: 1.0
              Discount Factor on Benefits: 0.970873786407767
              
+-----------------------------+
+------------------------------+


Total Costs and Coverage by Year
+------------------------------+


|   time |   opt_vals |      opt_benefit |        opt_costs |
|-------:|-----------:|-----------------:|-----------------:|
|      1 |          3 | 796827           | 599226           |
|      2 |          3 | 879986           | 644275           |
|      3 |          6 |      1.59095e+06 |      1.30343e+06 |
|      4 |         11 |      2.76946e+06 |      2.05228e+06 |
|      5 |          7 |      1.6732e+06  |      1.19193e+06 |
|      6 |          9 |      2.56e+06    |      1.84151e+06 |
|      7 |          9 |      2.33104e+06 |      1.60902e+06 |
|      8 |          8 |      2.15703e+06 |      1.44381e+06 |
|      9 |          7 |      2.04059e+06 |  


## Benefit Maximization

In [4]:
b = mm.BenefitSolver(data = df)

In [5]:
b.fit()


              Loading MIP Model with:
              Solver = CBC
              Method = MAX
              
[Note]: Optimal Solution Found


In [6]:
b.report()


              Optimized Scenario with:
              
              Method: MAX
              Discount Factor on Costs: 1.0
              Discount Factor on Benefits: 0.970873786407767
              
+-----------------------------+
+------------------------------+


Total Costs and Coverage by Year
+------------------------------+


|   time |   opt_vals |   opt_benefit |   opt_costs |
|-------:|-----------:|--------------:|------------:|
|      1 |         10 |   3.95692e+06 | 4.21231e+06 |
|      2 |         10 |   4.21901e+06 | 4.40096e+06 |
|      3 |          9 |   3.2952e+06  | 3.155e+06   |
|      4 |         19 |   7.06799e+06 | 6.80591e+06 |
|      5 |         10 |   3.23176e+06 | 2.87073e+06 |
|      6 |         16 |   5.24707e+06 | 4.67075e+06 |
|      7 |         14 |   4.1409e+06  | 3.40116e+06 |
|      8 |         13 |   3.32723e+06 | 2.51987e+06 |
|      9 |         13 |   3.37033e+06 | 2.54547e+06 |
|     10 |          7 |   1.79262e+06 | 1.23676e+06 |


+-------------